In [10]:
import tensorflow as tf
import numpy as np
import random
import os
from colorama import Fore
import locale

In [11]:
# Common Definitions
locale.setlocale(
    category=locale.LC_ALL,
    locale = '')

'English_United States.1252'

In [12]:
# بارگیری دیتاست
DataSet = ("mevlana", "molana_shams.txt")
with open(
        DataSet[1],
        mode = "r",
        encoding = "utf-8"
    ) as f:
    CleanText = f.read()

print(f"{Fore.WHITE}DataSet '{DataSet[0]}' has contains {Fore.GREEN}{len(Text):n}{Fore.WHITE} characters.")

DataSet 'mevlana' has contains 339,019 characters.


In [13]:
UniqueChars = sorted(set(CleanText))
# UniqueChars = " ءأئؤابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی"
Vocabulary = {char : index for index, char in enumerate(UniqueChars)}
CharacterMap = np.array(UniqueChars)
VocabularySize = len(Vocabulary)
print(f"{Fore.WHITE}DataSet '{DataSet[0]}' has contains {Fore.GREEN}{VocabularySize:n}{Fore.WHITE} unique characters:")
print(Vocabulary)

DataSet 'mevlana' has contains 35 unique characters:
{'\n': 0, ' ': 1, 'ا': 2, 'ب': 3, 'ت': 4, 'ث': 5, 'ج': 6, 'ح': 7, 'خ': 8, 'د': 9, 'ذ': 10, 'ر': 11, 'ز': 12, 'س': 13, 'ش': 14, 'ص': 15, 'ض': 16, 'ط': 17, 'ظ': 18, 'ع': 19, 'غ': 20, 'ف': 21, 'ق': 22, 'ل': 23, 'م': 24, 'ن': 25, 'ه': 26, 'و': 27, 'ٖ': 28, 'پ': 29, 'چ': 30, 'ژ': 31, 'ک': 32, 'گ': 33, 'ی': 34}


In [14]:
#Convert Text => Integer codded vector
CoddedText = np.array([Vocabulary[c] for c in CleanText if c in Vocabulary])
print(f"{CleanText[:74]}\n=>\n{CoddedText[:74]}")

ای رستخیز ناگهان وی رحمت بیمنتها
ای اتشی افروخته در بیشه اندیشهها

امروز خ
=>
[ 2 34  1 11 13  4  8 34 12  1 25  2 33 26  2 25  1 27 34  1 11  7 24  4
  1  3 34 24 25  4 26  2  0  2 34  1  2  4 14 34  1  2 21 11 27  8  4 26
  1  9 11  1  3 34 14 26  1  2 25  9 34 14 26 26  2  0  0  2 24 11 27 12
  1  8]


In [15]:
# تعریف داده‌های آموزشی و تست
SequenceLen = 100
ExamplesPerEpoch = len(CleanText) // SequenceLen

tsDataSet = tf.data.Dataset.from_tensor_slices(CoddedText)
Sequences = tsDataSet.batch(SequenceLen + 1, drop_remainder = True)

# تعریف تابع برای تبدیل داده به بردارهای آماده برای ورود به مدل
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

tsDataSet = Sequences.map(split_input_target)

In [16]:
# تعریف مدل
EmbeddingDimension = 256
RnningUnits = 1024
BatchSize = 64

def build_model(embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(VocabularySize, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(
            rnn_units,
            return_sequences = True,
            stateful = True,
            recurrent_initializer = 'glorot_uniform'),
        tf.keras.layers.Dense(VocabularySize)
    ])
    return model

model = build_model(EmbeddingDimension, RnningUnits, BatchSize)

In [17]:
# تعریف تابعی برای تولید شعر با استفاده از مدل
def generate_text(model, seed):
    # تعریف تعداد کاراکترهای تولیدی
    num_generate = 300

    # تبدیل کاراکترهای ورودی به بردار اعداد
    input_eval = [Vocabulary[s] for s in seed]
    input_eval = tf.expand_dims(input_eval, 0)

    # تعریف شعر تولیدی
    GeneratedText = []

    # دما برای تعیین تنوع در خروجی
    temperature = 1.0

    # تولید کاراکترهای جدید با استفاده از مدل LSTM
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        # تعیین احتمالات کاراکترهای بعدی با در نظر گرفتن دما
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples = 1)[-1,0].numpy()

        # ادام نتیجه شده را به شکل کاراکتر تبدیل کرده و به شعر تولیدی اضافه می‌کنیم
        GeneratedText.append(CharacterMap[predicted_id])

        # استفاده از کاراکتر تولید شده بعنوان ورودی برای کاراکتر بعدی
        input_eval = tf.expand_dims([predicted_id], 0)

    return (seed + ''.join(GeneratedText))


In [18]:
# آموزش مدل
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits = True)

model.compile(optimizer = 'adam', loss = loss)
checkpoint_dir = './Model'
checkpoint_prefix = os.path.join(checkpoint_dir, "epoch_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True)

history = model.fit(
    tsDataSet.batch(
        BatchSize,
        drop_remainder=True),
    epochs = 250,
    callbacks = [checkpoint_callback])

Epoch 1/250
52/52 [==============================] - 63s 1s/step - loss: 2.9648
Epoch 2/250
52/52 [==============================] - 63s 1s/step - loss: 2.5896
Epoch 3/250
52/52 [==============================] - 67s 1s/step - loss: 2.4675
Epoch 4/250
52/52 [==============================] - 68s 1s/step - loss: 2.3804
Epoch 5/250
52/52 [==============================] - 68s 1s/step - loss: 2.3083
Epoch 6/250
52/52 [==============================] - 69s 1s/step - loss: 2.2522
Epoch 7/250
52/52 [==============================] - 70s 1s/step - loss: 2.2040
Epoch 8/250
52/52 [==============================] - 68s 1s/step - loss: 2.1585
Epoch 9/250
52/52 [==============================] - 69s 1s/step - loss: 2.1159
Epoch 10/250
52/52 [==============================] - 68s 1s/step - loss: 2.0739
Epoch 11/250
52/52 [==============================] - 68s 1s/step - loss: 2.0331
Epoch 12/250
52/52 [==============================] - 68s 1s/step - loss: 1.9929
Epoch 13/250
52/52 [=================

In [19]:
# تولید شعر با استفاده از مدل آموزش داده شده
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(EmbeddingDimension, RnningUnits, batch_size = 1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [20]:
print(Vocabulary)

{'\n': 0, ' ': 1, 'ا': 2, 'ب': 3, 'ت': 4, 'ث': 5, 'ج': 6, 'ح': 7, 'خ': 8, 'د': 9, 'ذ': 10, 'ر': 11, 'ز': 12, 'س': 13, 'ش': 14, 'ص': 15, 'ض': 16, 'ط': 17, 'ظ': 18, 'ع': 19, 'غ': 20, 'ف': 21, 'ق': 22, 'ل': 23, 'م': 24, 'ن': 25, 'ه': 26, 'و': 27, 'ٖ': 28, 'پ': 29, 'چ': 30, 'ژ': 31, 'ک': 32, 'گ': 33, 'ی': 34}


In [21]:
print(CharacterMap)

['\n' ' ' 'ا' 'ب' 'ت' 'ث' 'ج' 'ح' 'خ' 'د' 'ذ' 'ر' 'ز' 'س' 'ش' 'ص' 'ض' 'ط'
 'ظ' 'ع' 'غ' 'ف' 'ق' 'ل' 'م' 'ن' 'ه' 'و' 'ٖ' 'پ' 'چ' 'ژ' 'ک' 'گ' 'ی']


In [24]:
Poem = generate_text(model, seed=u"شمس")
print(Poem)

شمست

امروز چنان خوان این جا

چنان کن که ز بد نالاست برگهست روید
من از این پاکتر از می جز ای شه عیسی و من
کردی در مرده ان زکار کی گفتم که چه میلوایهم ان چنان قضر
به هر دم بدیده استاد دشمن روان شود ز من مهمان شده تا بردی ای دم ضعا را

درکش قات اندر و او در فدا چرا

چون بدید ان شاه عقل و العش و شور
که شد


In [26]:
Poem = generate_text(model, seed=u"تبریزی")
print(Poem)

تبریزی به دست

جان میجویدم که مکان کیست

جهان بگشا طره و جانش که پی است سر اه کاسه را هست این جا

در ان جا بنگر که بود شمع تو شود
زان شاه که نیازم نه چون اندر دل ما بستان
که صد هزار منزل با قرار ماست

خود پیلوی یار کردی ای جو
ان مه سرایی تو که فروسیدم خوری
چون روی اه تو را این جهان بیستون را

نمود ان تا ی


In [27]:
Poem = generate_text(model, seed=u"سلطان")
print(Poem)

سلطان ما

در دو جهان لطیف و خوش همچو امیر ما کجا
ابروی او گره نشد گر چه که بیدار کند
بر خم در هستی در سرسرار ما
یک خوش امد و ن غین بترش ز ناعم گفتهای باران در طرب به جز این افراق نیست

در مذهر عنابه شوی کم عشق
کر من روان شدی دست در شکار نیست

گر بحر میپزید و دل بده مینشین منتظانست

ان در این عربده بمن هس


In [29]:
Poem = generate_text(model, seed=u"معشوق")
print(Poem)

معشوق مطرب مسکین که از تو چو گاه نیست

هر کس که دیده باده بلا و بر او براید خدا بود ای دخت طلب پاک
که بر از نفکی باشد که به زان برون
زخم این چه کارتست سایش غما

ای مست رقصان و میمان گشت ز اسمان ما

تا بدانی خار امد گر چه برجای چرا

ای برند چه بود هر روز چون گل
تا چه باشد اقبال بر عاشقان خم شده
است
که مرا


In [31]:
Poem = generate_text(model, seed=u"خاموش")
print(Poem)

خاموش باش مر مهار ما

چون بر سر کوی یار کردم به می دست کن
با منی همچون من که غیار خوش پنهان چیست

شمس تبریزی به خواندند بر رخ اندر دل پرو
زان کس که دریا در گوش تو اندر نباشد مرا

را چو من
گیری به من ندارم چه خوش بود به خدا

چو جان زار بلادیده با خدا گوید
که جز تو هیچ ندارد هر شب گشته یار بادا

به عشق این


In [32]:
Poem = generate_text(model, seed=u"نگار")
print(Poem)

نگاره خود را این نعره در او چون کفرین کیده
سیاهیت کاری نگرید این راه بیارگو و برون
در پی من هستی ما را مرا یاد ه ای برا

ای روز تو به دست برست

به گرد سنگ شیران شد سبق از ان دیده بار را

گر زان که نگر که نیایم چو کر و سینه برخاست

تو شاخی کن همچو بوی ای در و این بیجان
اهنی سرد را پیشتر از شکر ما

این هم


In [37]:
Poem = generate_text(model, seed=u"عاشق")
print(Poem)

عاشق ناگهانست

زین را که شیرین کند بر تو لاله داران را

چنین باشد ان نقص کشیده بدین که را
ای عقل باشد حال او خوش لقا

کوه طور و دفتم بیاباری را

بده بر خمشیره ندارم چه خوش بود به خدا

چو شیر پنجه نهد بر شکسته اهوی خویش
که ای عزیز شاد جان افرو و سازی شدست

تریا ندام عشق شمس دین
روز تو را و یفنی هر جا

هر


In [36]:
Poem = generate_text(model, seed=u"بهار")
print(Poem)

بهار بیداران ز کجا

طفل سلام سلطان و مستعل و تاسما

خاموش و تلقینا فی الکاف الصبا

لم ترزان ان در سر در و دینار اینست

من از میدان شمس دین ان قصه تو بیذاب

با ماییم ز تو چون زوسفین مایما

چون صباح مرد تهبیر شنی سرنا
بی تو در حجات مطلب گردوی ان سینه جان حریفان
جان یار که این خانه ت

دل ار دل بیشین خواجه 


In [ ]:
Poem = generate_text(model, seed=u"جان")
print(Poem)